<a href="https://colab.research.google.com/github/leard/notebooks/blob/master/Busca_de_Artigos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scholarly
!pip install free-proxy

In [1]:
from fp.fp import FreeProxy
from scholarly import scholarly
import pandas as pd

In [ ]:
search_term = '\"Virtual reality\" + \"EEG\" + \"EMG\" + Physiotherapy'
number_of_items = 200

labels = ['id', 'year', 'title', 'author', 'book_journal', 'type', 'organization', 'abstract_scholar', 'summary', 'cites', 'gsrank', 'citations_link','url','bibtex']
publications = []

scholarly.use_tor(tor_sock_port=9050, 
                  tor_control_port=9051, 
                  tor_password="12345")

search_query = scholarly.search_pubs(search_term)

for i, pub in enumerate(search_query):
    filled = None
    publication = []
    while True:
        try:
            filled = pub.fill()
            print(f"{i+1:03d} - {filled.bib['title']}")
            break
        except Exception as e:
            print(f'Tentando um novo proxy {e}')
            #set_new_proxy()
    if filled:
        publication.append(filled.bib['year'])
        publication.append(filled.bib['title'])
        publication.append(filled.bib['author'])        
        
        if filled.bib['ENTRYTYPE'] == 'article':
            try:
                publication.append(filled.bib['journal'])
            except:
                publication.append('')

        elif filled.bib['ENTRYTYPE'] == 'book' or filled.bib['ENTRYTYPE'] == 'misc':
            try:
                publication.append(filled.bib['publisher'])     
            except:
                publication.append('')
        else:
            publication.append(filled.bib['booktitle'])


        publication.append(filled.bib['ENTRYTYPE'])
        
        if filled.bib['ENTRYTYPE'] == 'article':
            try:
                publication.append(filled.bib['publisher'])     
            except:
                publication.append('')
        else:
            try:
                publication.append(filled.bib['organization'])
            except:
                publication.append('')
        
        
        try:
            publication.append(filled.bib['abstract'])
        except:
            publication.append('')

        publication.append('')
        publication.append(filled.bib['cites'])
        publication.append(filled.bib['gsrank'])
        
        try:
            publication.append('https://scholar.google.com'+filled.citations_link)
        except:
            publication.append('')

        
        try:
            publication.append(filled.bib['url'])
        except:
            publication.append('')

        publication.append(filled.bibtex)

    publications.append(publication)

    if i == number_of_items:
        break
    

publications = pd.DataFrame(publications, columns=labels)

In [ ]:
publications.head()

In [ ]:
#busca1 "Virtual reality" + "EEG" + "EMG" + Physiotherapy
publications.to_csv('./busca1.csv', index_label='id')

#busca2 "Virtual reality" + "EEG" + "EMG" + Rehabilitation
#publications.to_csv('./busca2.csv', index_label='id')